# pyDNA assembly objects

This notebook is looking into how to best utilize pyDNA `assembly` objects for Gibson assembly with fragments
that may or may not already contain homologous regions.

[Link to docs](https://pydna.readthedocs.io/#pydna.assembly.Assembly)



In [11]:
from pydna.assembly import Assembly
from pydna.dseqrecord import Dseqrecord

In [12]:
a = Dseqrecord("acgatgctatactgCCCCCtgtgctgtgctcta")
b = Dseqrecord("tgtgctgtgctctaTTTTTtattctggctgtatc")
c = Dseqrecord("tattctggctgtatcGGGGGtacgatgctatactg")

Limit is shortest shared homology that should be considered.

In [13]:
a = Assembly((a,b,c), limit=14)

In [14]:
a

Assembly
fragments..: 33bp 34bp 35bp
limit(bp)..: 14
G.nodes....: 6
algorithm..: common_sub_strings

In [15]:
a.assemble_circular()

[Contig(o59), Contig(o59)]

Does assembly work for overlapping ends?

In [18]:
d = Dseqrecord('aaaaaaaaaaaaatttttttttttttttGGGGG')
e = Dseqrecord('GGGGGcccccccccccccccccccccccccccc')

a1 = Assembly((d, e), limit=5)
a1.assemble_linear()
a1.assemble_linear()[0].seq.watson

aaaaaaaaaaaaatttttttttttttttGGGGGcccccccccccccccccccccccccccc

Expanding to a bit more complex case using random sequences.

In [22]:
def assemble_records(records, limit=10):
    return Assembly(records, limit).assemble_linear()[0].seq.watson

In [25]:
import numpy as np

def random_seq(n=25):
    return ''.join(np.random.choice(['a', 't', 'g', 'c'], n, replace=True))

rand_a = random_seq()
rand_b = random_seq()

homolgous_region = random_seq(10)
homolgous_region = homolgous_region.upper()

rand_a += homolgous_region
rand_b = homolgous_region + rand_b

rand_a = Dseqrecord(rand_a)
rand_b = Dseqrecord(rand_b)

In [27]:
print('Seq A:', rand_a.seq)
print('Seq B:', rand_b.seq)

Seq A: agtgcgcgacaatatcttagcagtaGTCGCAGTAC
Seq B: GTCGCAGTACccaatctttttaccgcatacttaaa


In [28]:
assemble_records((rand_a, rand_b))

agtgcgcgacaatatcttagcagtaGTCGCAGTACccaatctttttaccgcatacttaaa

Increasing the limit above 10 should result in no assembly.

In [31]:
try:
    assemble_records((rand_a, rand_b), 11)
except Exception as e:
    print(f'Assembly failed raised {e}')

Assembly failed raised list index out of range


What happens when no homology is present?

In [34]:
rand_c = Dseqrecord(random_seq(100))
rand_d = Dseqrecord(random_seq(100))

try:
    assemble_records((rand_c, rand_d), 15)
except Exception as e:
    print(f'Assembly failed raised {e}')

Assembly failed raised list index out of range


Linear assembly fails in this case which is good and this method can be used to ID fragments which would not require primers due 
preexisting homology. We also know the order in which the fragments are to be assembled so we can avoid doing a more
costly all against all type compariso between fragments for homology. 

Next step is figuring out how to implement this in an actual assembly when records are coming in as genbank files. Another question is if
DseqRecords can be assembled in this way alongside amplicons.